In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D

/usr/lib/python2.7/dist-packages/PIL/Image.py:71: RuntimeWarning: The _imaging extension was built for another  version of Pillow or PIL
  warnings.warn(str(v), RuntimeWarning)
/usr/lib/python2.7/dist-packages/PIL/Image.py:71: RuntimeWarning: The _imaging extension was built for another  version of Pillow or PIL
  warnings.warn(str(v), RuntimeWarning)
/usr/lib/python2.7/dist-packages/PIL/Image.py:71: RuntimeWarning: The _imaging extension was built for another  version of Pillow or PIL
  warnings.warn(str(v), RuntimeWarning)
/usr/lib/python2.7/dist-packages/PIL/Image.py:71: RuntimeWarning: The _imaging extension was built for another  version of Pillow or PIL
  warnings.warn(str(v), RuntimeWarning)


In [3]:
ld = open('/media/sf_kyoyu/data/complist_integer_compounds_pickup.csv')
lines = ld.readlines()
ld.close

delete_list = []
for i in ['He','Ne','Ar','Kr','Xe','Tc','Pm','Tb','Ne','Po','At','Rn','Fr','Ra','Ac','Th','Pa','U','Np','Pu','Am',\
          'Cm','Bk','Cf','Es','Fm','Md','No','Lr']:
    for line in lines:
        if line.find(i) >= 0:
            delete_list.append(line[:-1])
# print delete_list
compounds = []
for j in delete_list:
    compounds.append(j.split()[0])
# print compounds

In [4]:
atomicdata=pd.read_csv('/media/sf_kyoyu/data/atomic_data (all_data).csv')
atomicdata=atomicdata.set_index('Symbol')
atomicdata=atomicdata.drop(atomicdata.ix['Po':'Rg'].index).drop(['RareEarth','Nval','IE2','IE3','Cp-g','Cp-mol',\
                                                                 'E-vapor','ElectricalConductivity','MolerMagneticSusceptibility',\
                                                                'ThermalExpansion','Z','Nf','CuKa','MoKa'],axis=1).fillna(0)

complist_integer_compounds_pickup=pd.read_csv('/media/sf_kyoyu/data/complist_integer_compounds_pickup.csv',header=None,\
                                              delim_whitespace=True)
complist_integer_compounds_pickup.columns=["Compounds","H","He","Li","Be","B","C","N","O","F",\
                                           "Ne","Na","Mg","Al","Si","P","S","Cl","Ar","K","Ca","Sc",\
                                           "Ti","V","Cr","Mn","Fe","Co","Ni","Cu","Zn","Ga","Ge","As",\
                                           "Se","Br","Kr","Rb","Sr","Y","Zr","Nb","Mo","Tc","Ru","Rh",\
                                           "Pd","Ag","Cd","In","Sn","Sb","Te","I","Xe","Cs","Ba","La",\
                                          "Ce","Pr","Nd","Pm","Sm","Eu","Gd","Tb","Dy","Ho","Er","Tm",\
                                          "Yb","Lu","Hf","Ta","W","Re","Os","Ir","Pt","Au","Hg","Tl","Pb","Bi"]
complist_integer_compounds_pickup = complist_integer_compounds_pickup.set_index("Compounds")

# for i in compounds:
#     complist_integer_compounds_pickup=complist_integer_compounds_pickup.drop(i)
complist_integer_compounds_pickup = complist_integer_compounds_pickup[0:100]

In [5]:
# make average database
def make_average_database():
    datalist = []
    for i in np.array(complist_integer_compounds_pickup.index):
        data_list = []
        for k in np.array(atomicdata.columns):
            data = []
            x_list = []
            y_list = []
            for j in np.array(complist_integer_compounds_pickup.columns):
                if complist_integer_compounds_pickup.loc[i , j] != 0:
                    x = complist_integer_compounds_pickup.loc[i , j]
                    y = atomicdata.loc[j , k] * x
                    x_list.append(x)
                    y_list.append(y)
                else : continue
            z = sum(y_list) / sum(x_list)
            data.append(z)
            data_list.append(data)
        datalist.append(data_list)
    frame = pd.DataFrame(datalist, columns = atomicdata.columns, index = complist_integer_compounds_pickup.index)
    frame.to_csv('/media/sf_kyoyu/data/average_database.csv')

In [2]:
average_data = pd.read_csv("/media/sf_kyoyu/data/average_database_ICSD_重複なし.csv").set_index("Compounds").\
drop(['Z','Nf','CuKa','MoKa'], axis=1)
average_data = average_data[0:100]
C1 = average_data

In [3]:
# def make_average_database():
A = complist_integer_compounds_pickup
B = atomicdata
C1 = A.dot(B)
for i in range(len(A.index)):
    C1.iloc[i , :] = C1.iloc[i , :]/(A.sum(axis=1)[i])
#     print C1

NameError: name 'complist_integer_compounds_pickup' is not defined

In [4]:
# 平均ー逆数データ
C1_inverse = 1/C1
C1_inverse = C1_inverse.replace('inf',0)

# 平均ー二乗データ
C1_square = C1*C1

# 平均ー対数データ
C1_log = np.log(C1)
C1_log = C1_log.replace('-inf',0).fillna(0)

In [5]:
# 平均データの標準化
scaled_list = []
for i in np.array(C1.columns):
    sc = StandardScaler()
    sc.fit(C1.loc[:,i])
    array_scaled = sc.transform(C1.loc[:,i])
#     print array_scaled
    scaled_list.append(array_scaled)
df = pd.DataFrame(scaled_list).T
df.index = np.array(C1.index)
df.columns = np.array(C1.columns)

# 平均ー逆数データの標準化
scaled_list_inverse = []
for i in np.array(C1_inverse.columns):
    sc = StandardScaler()
    sc.fit(C1_inverse.loc[:,i])
    array_scaled_inverse = sc.transform(C1_inverse.loc[:,i])
    scaled_list_inverse.append(array_scaled_inverse)
df_inverse = pd.DataFrame(scaled_list_inverse).T
df_inverse.index = np.array(C1_inverse.index)
df_inverse.columns = np.array(C1_inverse.columns)

# 平均ー二乗データの標準化
scaled_list_square = []
for i in np.array(C1_square.columns):
    sc = StandardScaler()
    sc.fit(C1_square.loc[:,i])
    array_scaled_square = sc.transform(C1_square.loc[:,i])
    scaled_list_square.append(array_scaled_square)
df_square = pd.DataFrame(scaled_list_square).T
df_square.index = np.array(C1_square.index)
df_square.columns = np.array(C1_square.columns)

# 平均ー対数データの標準化
scaled_list_log = []
for i in np.array(C1_log.columns):
    sc = StandardScaler()
    sc.fit(C1_log.loc[:,i])
    array_scaled_log = sc.transform(C1_log.loc[:,i])
    scaled_list_log.append(array_scaled_log)
df_log = pd.DataFrame(scaled_list_log).T
df_log.index = np.array(C1_log.index)
df_log.columns = np.array(C1_log.columns)

# 平均ー全データ連結
df_all = pd.concat([df, df_inverse, df_square, df_log], axis=1)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.re

In [6]:
# 主成分分析
pca = PCA(n_components = 3, whiten = False)
pca.fit(df)
df_pca = pca.transform(df)
E = pca.explained_variance_ratio_
df_pca = pd.DataFrame(df_pca, columns = ["第一主成分","第二主成分","第三主成分"], index = df.index)
print pca.components_
print np.cumsum(E)

[[-0.26729305  0.23665536  0.18210314  0.28733377 -0.20451096 -0.2338135
   0.30536641  0.23457827  0.30117118 -0.26994538 -0.28899403 -0.25405391
  -0.2651391  -0.22418258 -0.18231909 -0.21592818]
 [ 0.3491362   0.32186397  0.05161743  0.26339842  0.36511787  0.41231681
   0.0937717   0.29830935  0.03305673  0.19995804  0.14286598 -0.22534568
  -0.20504221  0.33061812 -0.07468368 -0.17936265]
 [-0.11210044  0.25084987 -0.21091071 -0.01812296  0.2620961  -0.03715035
   0.23494848  0.14772923  0.22985718 -0.27551598 -0.31303855  0.26599393
   0.17588392  0.33764372  0.3894949   0.37210594]]
[ 0.57585213  0.72975399  0.82657232]


In [10]:
# 二次元プロット
plt.scatter(df_pca.ix[:,0], df_pca.ix[:,1])
plt.show()

In [9]:
# 三次元プロット
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter3D(df_pca.ix[:,0], df_pca.ix[:,1], df_pca.ix[:,2])
plt.show()

/usr/lib/python2.7/dist-packages/matplotlib/collections.py:571: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
/usr/lib/python2.7/dist-packages/matplotlib/pyplot.py:424: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/usr/lib/python2.7/dist-packages/mpl_toolkits/mplot3d/axes3d.py:1094: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self.button_pressed in self._rotate_btn:
